In [ ]:
import requests
import json
import urllib.request
import time
import pandas as pd

# 헤더 저장(자신의 로그인 정보를 넣으면 됩니다!)
headers = {
            'cookie' : '',
            'user-agent': ''
            }

# 해시 태그 설정
tags = "20대"
user_list = []
count = 0

#첫 페이지
res = requests.get('https://www.instagram.com/explore/tags/{}/?__a=1'.format(tags), headers = headers)
res_dics = json.loads(res.text)
#print(res_dics)
if len(res_dics) > 0:
  if res_dics['graphql']['hashtag']["edge_hashtag_to_media"]["page_info"]["has_next_page"] == True:
    first_end =  res_dics['graphql']['hashtag']["edge_hashtag_to_media"]["page_info"]["end_cursor"]
    #print(first_end)
    end_cursor = first_end
    isTrue = True

    while isTrue:
      next_url = 'https://www.instagram.com/graphql/query/?query_hash=c769cb6c71b24c8a86590b22402fda50&variables=%7B"tag_name"%3A"{}"%2C"first"%3A2%2C"after"%3A"{}"%7D'.format(tags, end_cursor)
      res2 = requests.get(next_url)
      res2_dics = json.loads(res2.text)

      if len(res2_dics) > 0:

        if len(res2_dics['data']['hashtag']["edge_hashtag_to_media"]['edges']) > 0:
          for i in res2_dics['data']['hashtag']["edge_hashtag_to_media"]['edges']:
            count += 1
            if count%300 == 0:
              print(count)
              time.sleep(300)
              print()
            user_list.append(i['node']['owner']['id'])
        else:
          print('해당 json 파일에 게시글 정보가 없습니다.')

        if res2_dics['data']['hashtag']["edge_hashtag_to_media"]["page_info"]["has_next_page"] == True:
          end_cursor =  res2_dics['data']['hashtag']["edge_hashtag_to_media"]["page_info"]["end_cursor"]  
          print(end_cursor) 
        else:
          isTrue = False
else:
  print('해당 태그와 관련된 게시물이 없습니다.')

print(user_list)

#user_list csv파일로 저장
unique_user_list = list(set(user_list))
dataframe = pd.DataFrame(unique_user_list)
dataframe.to_csv("csv파일을 저장할 폴더 경로/{}.csv".format(tags), header=False, index = False)  #첫번째 파라미터는 저장할 csv파일을 저장할 폴더 경로로 바꿔 넣으면 됩니다!
